In [3]:
import json
import os
from linchemin.cheminfo.constructors import MoleculeConstructor
from linchemin.cgu.translate import translator
from tqdm import tqdm
from noctis.repository.neo4j.neo4j_repository import Neo4jRepository
import time
from concurrent.futures import ThreadPoolExecutor, TimeoutError, as_completed
import itertools
from linchemin.rem.route_descriptors import descriptor_calculator
from linchemin.cgu.syngraph_operations import extract_reactions_from_syngraph
from pandas import DataFrame


### Execution of custom queries [Table 3]

In [12]:
repo = Neo4jRepository( database='uspto')

In [72]:
### Uncomment if you want to reload the USPTO dataset

# IMPORT_PATH = r'C:\Users\s1222744\Neo4jDesktopDB\relate-data\dbmss\dbms-3b486baa-cae1-4147-8a3d-d7fd445cdede\import'
# If your Neo4j is set up to have an import directory,
# first move the files to the import directory before running this query
# If your Neo4j is set up to import files from anywhere in your file system, define the import path

# repo.execute_custom_query_from_yaml(yaml_file = "./queries/table3.yaml", query_name = 'delete_all_nodes')
# repo.execute_query(query_name = 'import_db_from_csv', folder_path = IMPORT_PATH, prefix = 'USPTO')


2025-04-07 16:56:38,090 - noctis.repository.neo4j.neo4j_repository - WARNING - Nodes and/or relationships are not defined in the GDB schema DataFrame.


[<Record file='progress.csv' source='file' format='csv' nodes=1080847 relationships=1336500 properties=3951533 time=86995 rows=0 batchSize=-1 batches=0 done=True data=None>,
 <Record rel=<Relationship element_id='5:ae64bf75-0648-4494-b940-76395c96410e:9' nodes=(<Node element_id='4:ae64bf75-0648-4494-b940-76395c96410e:521443' labels=frozenset() properties={}>, <Node element_id='4:ae64bf75-0648-4494-b940-76395c96410e:32433' labels=frozenset() properties={}>) type='REACTANT' properties={'__csv_type': 'REACTANT'}>>,
 <Record rel=<Relationship element_id='5:ae64bf75-0648-4494-b940-76395c96410e:10' nodes=(<Node element_id='4:ae64bf75-0648-4494-b940-76395c96410e:465556' labels=frozenset() properties={}>, <Node element_id='4:ae64bf75-0648-4494-b940-76395c96410e:32433' labels=frozenset() properties={}>) type='REACTANT' properties={'__csv_type': 'REACTANT'}>>,
 <Record rel=<Relationship element_id='5:ae64bf75-0648-4494-b940-76395c96410e:25' nodes=(<Node element_id='4:ae64bf75-0648-4494-b940-7639

In [78]:

queries = ['count_root_nodes', 'count_leaf_nodes', 'average_reactant_count', 'average_product_to_molecule_ratio', 'molecules_with_multiple_products']

# Dictionary to map query names to more readable metric names
metric_names = {
    'count_root_nodes': 'Root Molecules',
    'count_leaf_nodes': 'Leaf Molecules',
    'average_reactant_count': 'Avg. Reactants per Equation',
    'average_product_to_molecule_ratio': 'Products-to-Molecules Ratio',
    'molecules_with_multiple_products': 'OR Nodes'
}

# List to store results
results = []

for query_name in queries:
    result_df = repo.execute_custom_query_from_yaml(yaml_file="./queries/table3.yaml", query_name=query_name)
    # Assuming the result is a DataFrame with a single value. Adjust if the structure is different.
    value = result_df.iloc[0, 0]
    
    # Round float values to 2 decimal places
    if isinstance(value, float):
        value = round(value, 1)
    
    results.append([metric_names[query_name], value])

# Create DataFrame
df = DataFrame(results, columns=['Metric', 'Value'])

In [79]:
print(df)

                        Metric     Value
0               Root Molecules  308010.0
1               Leaf Molecules  188614.0
2  Avg. Reactants per Equation       1.8
3  Products-to-Molecules Ratio       2.5
4                     OR Nodes   12347.0


### Paracetamol example [Table 1]

In [5]:
repo = Neo4jRepository(database='uspto')

In [6]:
def process_target(repo, target, max_chemical_equations=None, dict_smiles_max_length=None, blacklist_smiles=None):
    if blacklist_smiles and target in blacklist_smiles:
        return 'blacklisted', target, None, None

    start_time = time.time()
    
    if dict_smiles_max_length:
        max_chemical_equations = dict_smiles_max_length.get(target, max_chemical_equations)

    node_by_smiles = repo.execute_custom_query_from_yaml(yaml_file='queries/table1.yaml', query_name='get_node_by_smiles', smiles=target)
    
    if not node_by_smiles.records:
        return 'not_found', target, None, None

    node_uid = node_by_smiles.records[0].nodes[0].uid
    try:
        if max_chemical_equations:
            dc = repo.execute_query(query_name='get_routes', root_node_uid=node_uid, max_number_reactions=max_chemical_equations)
        else:
            dc = repo.execute_query(query_name='get_routes', root_node_uid=node_uid)
        
        processing_time = time.time() - start_time
        return 'success', target, dc, processing_time
    except Exception as e:
        return 'error', target, str(e), None

In [7]:
dict_of_dc = {}
smiles_not_found = []
timed_out_smiles = []
error_smiles = []
targets = ['CC(=O)Nc1ccc(O)cc1']
max_number_reactions = 4
max_total_time = 300  

with ThreadPoolExecutor(max_workers=5) as executor:
    future_to_target = {executor.submit(process_target, repo, target, max_number_reactions, None, None ): target for target in targets}
    
    start_time = time.time()
    for future in as_completed(future_to_target):
        if time.time() - start_time > max_total_time:
            print("Total time exceeded. Stopping processing.")
            unprocessed_smiles = [target for target, future in future_to_target.items() if not future.done()]
            print(f"Unprocessed SMILES due to timeout: {unprocessed_smiles}")
            break

        target = future_to_target[future]
        try:
            result = future.result(timeout=20)  # 20 seconds timeout per target
            status, smiles, data, processing_time = result
            if status == 'success':
                dict_of_dc[target] =data.transform_to(format_type = 'syngraph')
                print(f'All good: {smiles}. Number of routes found: {len(data.records)}. Time taken: {processing_time:.2f} seconds.')
            elif status == 'not_found':
                smiles_not_found.append(smiles)
                #print(f'Not found: {smiles}')
            elif status == 'blacklisted':
                print(f'Skipped (blacklisted): {smiles}')
            else:
                error_smiles.append(smiles)
                print(f'Error processing {smiles}: {data}')
        except TimeoutError:
            timed_out_smiles.append(target)
            print(f'Timeout: {target}')

print(f"Processed {len(dict_of_dc.keys())} targets successfully")
print(f"Targets not found: {len(smiles_not_found)}")
print(f"Targets timed out: {len(timed_out_smiles)}")
print(f"Targets with errors: {len(error_smiles)}")

All good: CC(=O)Nc1ccc(O)cc1. Number of routes found: 99. Time taken: 2.67 seconds.
Processed 1 targets successfully
Targets not found: 0
Targets timed out: 0
Targets with errors: 0


In [8]:
# Paracetamol
investigated_smiles = 'CC(=O)Nc1ccc(O)cc1'
list_long_seq = []
list_nr_branches = []
list_nr_steps = []
for route in dict_of_dc[investigated_smiles]:
    ls = descriptor_calculator(route, "longest_seq")
    nbr = descriptor_calculator(route, "nr_branches")
    nstp = len(set(item['input_string'] for item in extract_reactions_from_syngraph(route)))
    list_long_seq.append(ls)
    list_nr_branches.append(nbr)
    list_nr_steps.append(nstp)
    
    print(f"Longest sequence: {ls}, number of branches: {nbr}, number of steps: {nstp}")

Longest sequence: 4, number of branches: 5, number of steps: 6
Longest sequence: 4, number of branches: 5, number of steps: 6
Longest sequence: 4, number of branches: 5, number of steps: 6
Longest sequence: 4, number of branches: 5, number of steps: 6
Longest sequence: 4, number of branches: 6, number of steps: 7
Longest sequence: 4, number of branches: 5, number of steps: 6
Longest sequence: 4, number of branches: 5, number of steps: 6
Longest sequence: 4, number of branches: 5, number of steps: 6
Longest sequence: 4, number of branches: 5, number of steps: 6
Longest sequence: 3, number of branches: 4, number of steps: 5
Longest sequence: 4, number of branches: 5, number of steps: 6
Longest sequence: 4, number of branches: 5, number of steps: 6
Longest sequence: 4, number of branches: 5, number of steps: 6
Longest sequence: 4, number of branches: 5, number of steps: 6
Longest sequence: 4, number of branches: 5, number of steps: 6
Longest sequence: 4, number of branches: 5, number of s

In [11]:

def create_spread_string_and_average(numbers):
    if not numbers:
        return None  # Return None for an empty list

    minimum = min(numbers)
    maximum = max(numbers)
    string = f"{minimum}-{maximum}"
    average = sum(numbers)/len(numbers)

    return string, average

number_of_routes = len(dict_of_dc[investigated_smiles])
longest_seq, avg_longest_seq = create_spread_string_and_average(list_long_seq)
n_steps, avg_n_steps = create_spread_string_and_average(list_nr_steps)
n_branches, avg_n_branches = create_spread_string_and_average(list_nr_branches)

print(f"N_route: {number_of_routes}, Longest seq: {longest_seq} with average {avg_longest_seq}, N branches: {n_branches} with average {avg_n_steps}, N steps: {n_steps} with average {avg_n_branches}")


N_route: 99, Longest seq: 0-4 with average 3.6363636363636362, N branches: 2-6 with average 5.808080808080808, N steps: 3-7 with average 4.787878787878788


In [10]:
dataframe_prodCount_distinct_smiles = repo.execute_custom_query_from_yaml(yaml_file = './queries/table1.yaml', query_name='products_per_distinct_molecule', smiles = 'CC(=O)Nc1ccc(O)cc1')
prodC = dataframe_prodCount_distinct_smiles['productCount']
distinct_smiles = dataframe_prodCount_distinct_smiles['distinct_smiles']
# removed the last, because they have a single product relationship
avg_prod_count = 0
n_distinct_smiles = 0
for i in range(len(prodC[:-1].to_list())):
    avg_prod_count += int(prodC[:-1].to_list()[i])*int(distinct_smiles[:-1].to_list()[i])
    n_distinct_smiles += int(distinct_smiles[:-1].to_list()[i])
avg_prod_count = avg_prod_count/n_distinct_smiles
spread_prod_count_string = f'{min(prodC[:-1].to_list())}-{max(prodC[:-1].to_list())}'
print(f"Average number of products per OR nodes: {avg_prod_count}, min/max product per OR node: {spread_prod_count_string}, number of OR nodes: {n_distinct_smiles}")
    

Average number of products per OR nodes: 4.222222222222222, min/max product per OR node: 2-10, number of OR nodes: 9


In [13]:
# Create a dictionary with the data


data = {
    "Metric": [
        "Target Compound",
        "Smiles",
        "Number of Routes",
        "Longest Sequence",
        "Number of Steps",
        "Number of Branches",
        "Number of OR nodes*",
        "Number of Products per OR node*"
    ],
    "Value": [
        'Paracetamol',
        investigated_smiles,
        number_of_routes,
        longest_seq,
        n_steps,
        n_branches,
        n_distinct_smiles,
        spread_prod_count_string
        
    ],
    "Avg. Value":[
        '',
        '',
        '',
        avg_longest_seq,
        avg_n_steps,
        avg_n_branches,
        '',
        avg_prod_count
    ]
}

# Create a DataFrame
df = DataFrame(data)
print(df)

                            Metric               Value Avg. Value
0                  Target Compound         Paracetamol           
1                           Smiles  CC(=O)Nc1ccc(O)cc1           
2                 Number of Routes                  99           
3                 Longest Sequence                 0-4   3.636364
4                  Number of Steps                 3-7   5.808081
5               Number of Branches                 2-6   4.787879
6              Number of OR nodes*                   9           
7  Number of Products per OR node*                2-10   4.222222


### Preparation for Table 2

In [92]:
PaRoute_PATH = '../data/paroute'
N1_ROUTES = os.path.join(PaRoute_PATH, 'n1-routes.json')
N1_TARGETS = os.path.join(PaRoute_PATH, 'n1-targets.txt')
N5_ROUTES = os.path.join(PaRoute_PATH, 'n5-routes.json')
N5_TARGETS = os.path.join(PaRoute_PATH, 'n5-targets.txt')

### Please follow the instructions in https://pubs.rsc.org/en/content/articlelanding/2022/dd/d2dd00015f PaRoutes to get the corresponding data

In [93]:
# Load targets

n1_targets_list = []
with open(N1_TARGETS, 'r') as file:
    for line in file:
        n1_targets_list.append(line.strip())

n5_targets_list = []
with open(N5_TARGETS, 'r') as file:
    for line in file:
        n5_targets_list.append(line.strip())

In [94]:
print(f"The number of n1 targets: {len(n1_targets_list)}")
print(f"The number of n5 targets: {len(n5_targets_list)}")

The number of n1 targets: 10000
The number of n5 targets: 10000


In [95]:
def have_common_elements(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    return bool(set1 & set2)


list1 = n1_targets_list
list2 = n5_targets_list

if have_common_elements(list1, list2):
    print("The lists have common elements.")
else:
    print("The lists do not have common elements.")

# To find the common elements:
common_elements = set(list1) & set(list2)
print("Common elements:", len(common_elements))

The lists have common elements.
Common elements: 2642


In [96]:
n1_routes = json.loads(open(N1_ROUTES).read())
n5_routes = json.loads(open(N5_ROUTES).read())


In [97]:
def get_shared_routes(list_of_routes, list_of_common_elements):
    result = {}
    mol_constructor = MoleculeConstructor()
    
    for route in list_of_routes:
        
        if route['smiles'] in list_of_common_elements:
            canonical_smiles = mol_constructor.build_from_molecule_string(route['smiles'], "smiles")
            if canonical_smiles.smiles not in result:
                result[canonical_smiles.smiles] = []
            result[canonical_smiles.smiles].append(route)
    
    return result

In [98]:
n1_and_n5_routes = n1_routes + n5_routes
routes_for_common_targets = get_shared_routes(n1_and_n5_routes, common_elements)

In [99]:
len(routes_for_common_targets['Cc1cccc2nc(C(C)N)c(-c3ccccn3)c(=O)n12'])

2

In [102]:
synroutes_for_common_targets = {}

for canonical_smiles in tqdm(itertools.islice(routes_for_common_targets.keys(), 100), desc="Processing SMILES"):
    synroutes_for_common_targets[canonical_smiles] = []
    
    for az_route in routes_for_common_targets[canonical_smiles]:
        syngraph = translator('az_retro', az_route, 'syngraph', 'bipartite')
        synroutes_for_common_targets[canonical_smiles].append(syngraph)

Processing SMILES: 100it [00:16,  5.99it/s]


In [103]:
### We check that for each target all the routes in PaRoutes are the same
different_routes = []
for smiles, routes in synroutes_for_common_targets.items():
    if routes[0] != routes[1]:
        different_routes.append(smiles)

print(f'For {len(different_routes)} selected smiles the AZ routes are not the same in n1 and n5')

For 0 selected smiles the AZ routes are not the same in n1 and n5


In [104]:
# We save the longest linear sequence of AZ routes in a dictionary to mine later the routes of the same length
count = 0
dict_canonical_smiles_longest_sequence = {}
for canonical_smiles, syngraphs in synroutes_for_common_targets.items():
    syngraph0 = syngraphs[0]
    syngraph1 = syngraphs[1]
    ls = descriptor_calculator(syngraph0, "longest_seq")
    nbr = descriptor_calculator(syngraph0, "nr_branches")
    print(f"{count} smiles: {canonical_smiles}, longest sequence: {ls}, number of branches: {nbr}")
    dict_canonical_smiles_longest_sequence[canonical_smiles] = descriptor_calculator(syngraph1, "longest_seq")
    count+=1


0 smiles: Cc1cccc2nc(C(C)N)c(-c3ccccn3)c(=O)n12, longest sequence: 9, number of branches: 0
1 smiles: CCN(C)Cc1cc2c(F)cccc2nc1CNc1ncnc(N)c1C#N, longest sequence: 9, number of branches: 0
2 smiles: Cc1ccc(CC(=O)O)c(O)c1, longest sequence: 6, number of branches: 0
3 smiles: CN(C)C(=O)c1cc(F)cc(C(C2CN(C(c3ccc(Cl)cc3)c3cccc(C#N)c3)C2)C(C)(C)F)c1, longest sequence: 7, number of branches: 0
4 smiles: CCC(=O)Nc1cncc(-c2cnc3c(c2)c(C=O)nn3C2CCCCO2)c1, longest sequence: 7, number of branches: 0
5 smiles: COc1ccc(-c2ncc(F)c(N(C)CCCOc3ccc4c(ccn4CC(=O)O)c3)n2)cc1, longest sequence: 7, number of branches: 0
6 smiles: CCCCCCCCCCCCCCCCNc1ccc(C(=O)CS(=O)(=O)c2ccccc2)cc1F, longest sequence: 6, number of branches: 0
7 smiles: CCCN(CCC)CCCCc1cc(Cl)ccc1OCCc1ccccc1, longest sequence: 10, number of branches: 0
8 smiles: N#Cc1cc(Cl)c(C(=O)c2c[nH]c3cnccc23)c(Cl)c1, longest sequence: 6, number of branches: 0
9 smiles: COc1ccc(CN2CC(C(C)(C)S(=O)(=O)c3cccc(C(F)(F)F)c3)CC2=O)cc1, longest sequence: 7, number of bra

### Comparison to PaRoute dataset [Table 2]

Because some of the targets are not present in MIT UPSTO, we add the routes for some selected targets to USPTO dataset
for the sake of comparing the routes mined with Noctis. 
We chose to select among the targets which are shared among n1 set and n5 set.
We investigated first 10 targets and we took one target with branching route.

In [105]:
### adding first 10 targets shared between n1 and n5
repo.execute_query(query_name = 'load_nodes_and_relationships', data = [synroutes_for_common_targets[smls][0] for smls in  itertools.islice(routes_for_common_targets.keys(), 10)], data_type = 'syngraph', output_chem_format = 'smiles')

[]

In [106]:
### adding a target with branching route
repo.execute_query(query_name = 'load_nodes_and_relationships', data = [synroutes_for_common_targets[smls][0] for smls in ['CCC(O)(/C=C/c1ccc(C(CC)(CC)c2cc(C)c(-c3ccc(CC(=O)O)c(F)c3)c(C)c2)cc1C)CC']], data_type = 'syngraph', output_chem_format = 'smiles')

[]

In [107]:
dict_of_dc = {}
smiles_not_found = []
timed_out_smiles = []
error_smiles = []
# These are smiles for which route mining explodes due to large number of mined routes
blacklisted_smiles = ['CCC(=O)Nc1cncc(-c2cnc3c(c2)c(C=O)nn3C2CCCCO2)c1', 'CCCCCCCCCCCCCCCCNc1ccc(C(=O)CS(=O)(=O)c2ccccc2)cc1F', 'CCCN(CCC)CCCCc1cc(Cl)ccc1OCCc1ccccc1', 'Cc1cccc2nc(C(C)N)c(-c3ccccn3)c(=O)n12', 'CCN(C)Cc1cc2c(F)cccc2nc1CNc1ncnc(N)c1C#N']
targets = list(itertools.islice(routes_for_common_targets.keys(), 10) )
targets.append('CCC(O)(/C=C/c1ccc(C(CC)(CC)c2cc(C)c(-c3ccc(CC(=O)O)c(F)c3)c(C)c2)cc1C)CC')
max_total_time = 300  # 5 minutes total timeout


with ThreadPoolExecutor(max_workers=5) as executor:
    future_to_target = {executor.submit(process_target, repo, target,None, dict_canonical_smiles_longest_sequence, blacklisted_smiles): target for target in targets}
    
    start_time = time.time()
    for future in as_completed(future_to_target):
        if time.time() - start_time > max_total_time:
            print("Total time exceeded. Stopping processing.")
            unprocessed_smiles = [target for target, future in future_to_target.items() if not future.done()]
            print(f"Unprocessed SMILES due to timeout: {unprocessed_smiles}")
            break

        target = future_to_target[future]
        try:
            result = future.result(timeout=20)  # 20 seconds timeout per target
            status, smiles, data, processing_time = result
            if status == 'success':
                dict_of_dc[target] =data.transform_to(format_type = 'syngraph')
                print(f'All good: {smiles}. Number of routes found: {len(data.records)}. Time taken: {processing_time:.2f} seconds. Length in PaRoutes: {dict_canonical_smiles_longest_sequence[smiles]}')
            elif status == 'not_found':
                smiles_not_found.append(smiles)
                #print(f'Not found: {smiles}')
            elif status == 'blacklisted':
                print(f'Skipped (blacklisted): {smiles}')
            else:
                error_smiles.append(smiles)
                print(f'Error processing {smiles}: {data}')
        except TimeoutError:
            timed_out_smiles.append(target)
            print(f'Timeout: {target}')

print(f"Processed {len(dict_of_dc.keys())} targets successfully")
print(f"Targets not found: {len(smiles_not_found)}")
print(f"Targets timed out: {len(timed_out_smiles)}")
print(f"Targets with errors: {len(error_smiles)}")
print(f"Targets blacklisted: {len(blacklisted_smiles)}")

Skipped (blacklisted): CCCN(CCC)CCCCc1cc(Cl)ccc1OCCc1ccccc1
Skipped (blacklisted): CCC(=O)Nc1cncc(-c2cnc3c(c2)c(C=O)nn3C2CCCCO2)c1
Skipped (blacklisted): CCN(C)Cc1cc2c(F)cccc2nc1CNc1ncnc(N)c1C#N
Skipped (blacklisted): CCCCCCCCCCCCCCCCNc1ccc(C(=O)CS(=O)(=O)c2ccccc2)cc1F
Skipped (blacklisted): Cc1cccc2nc(C(C)N)c(-c3ccccn3)c(=O)n12
All good: Cc1ccc(CC(=O)O)c(O)c1. Number of routes found: 6. Time taken: 1.10 seconds. Length in PaRoutes: 6
All good: CCC(O)(/C=C/c1ccc(C(CC)(CC)c2cc(C)c(-c3ccc(CC(=O)O)c(F)c3)c(C)c2)cc1C)CC. Number of routes found: 270. Time taken: 1.85 seconds. Length in PaRoutes: 6
All good: COc1ccc(-c2ncc(F)c(N(C)CCCOc3ccc4c(ccn4CC(=O)O)c3)n2)cc1. Number of routes found: 3. Time taken: 3.22 seconds. Length in PaRoutes: 7
All good: N#Cc1cc(Cl)c(C(=O)c2c[nH]c3cnccc23)c(Cl)c1. Number of routes found: 3. Time taken: 3.40 seconds. Length in PaRoutes: 6
All good: CN(C)C(=O)c1cc(F)cc(C(C2CN(C(c3ccc(Cl)cc3)c3cccc(C#N)c3)C2)C(C)(C)F)c1. Number of routes found: 11. Time taken: 3.58 s

In [108]:
# a function to check if AZ routes is a subset to Boctis routes
def find_shared_reactions_between_two_syngraphs(obj1, obj2):
    input_strings1 = set(item['input_string'] for item in extract_reactions_from_syngraph(obj1))
    input_strings2 = set(item['input_string'] for item in extract_reactions_from_syngraph(obj2))

    # Find similar input_strings
    similar_strings = input_strings1.intersection(input_strings2)

    # Count of similar strings
    count_similar = len(similar_strings)

    return similar_strings, count_similar, len(input_strings1), len(input_strings2)

In [112]:
exact_match = {}
shared_reactions = {}
steps_az = {}
steps_ncts = {}
long_seq_az = {}
long_seq_ncts = {}
branch_az = {}
branch_ncts = {}
routes_which_donot_contain_some_reactions= {}
for smiles, noctis_syngraphs in dict_of_dc.items():
    exact_match[smiles] = []
    shared_reactions[smiles] = []
    steps_az[smiles] = []
    steps_ncts[smiles] = []
    long_seq_az[smiles] = []
    long_seq_ncts[smiles] = []
    branch_az[smiles] = []
    branch_ncts[smiles] = []
    routes_which_donot_contain_some_reactions[smiles] = []
    print(f"{smiles}, N route mined with Noctis: {len(noctis_syngraphs)} ----------------------")
    for i, ncts_sng in enumerate(noctis_syngraphs):
        
        az_sng = synroutes_for_common_targets[smiles][0]
        print(f"{i} Exact match Noctis and AZ routes: {ncts_sng == az_sng}")
        exact_match[smiles].append(ncts_sng == az_sng)
        r, nr_shared, stp_az, stp_ncts = find_shared_reactions_between_two_syngraphs(az_sng,ncts_sng)
        az_ls = descriptor_calculator(az_sng, "longest_seq")
        ncts_ls = descriptor_calculator(ncts_sng, "longest_seq")
        az_nrb = descriptor_calculator(az_sng, "nr_branches")
        ncts_nrb = descriptor_calculator(ncts_sng, "nr_branches")
        shared_reactions[smiles].append(nr_shared)
        steps_az[smiles] = [stp_az]
        steps_ncts[smiles].append(stp_ncts)
        long_seq_az[smiles] = [az_ls]
        long_seq_ncts[smiles] = [ncts_ls]
        branch_az[smiles] = [az_nrb]
        branch_ncts[smiles] = [ncts_nrb]
        print(f"N reactions shared: {nr_shared}, N steps in AZ route: {stp_az}, in Noctis route: {stp_ncts}")
        if nr_shared != stp_az:
            routes_which_donot_contain_some_reactions[smiles].append(i)
        print(f"Longest sequence in AZ route: {az_ls}, in Noctis route: {ncts_ls}")
        print(f"N branches in AZ route: {az_nrb}, in Noctis route: {ncts_nrb}")
    print("")

    print(f"Routes which do not contain some reactions from AZ route: {len(routes_which_donot_contain_some_reactions[smiles])}")
    

Cc1ccc(CC(=O)O)c(O)c1, N route mined with Noctis: 6 ----------------------
0 Exact match Noctis and AZ routes: False
N reactions shared: 6, N steps in AZ route: 6, in Noctis route: 7
Longest sequence in AZ route: 6, in Noctis route: 6
N branches in AZ route: 0, in Noctis route: 2
1 Exact match Noctis and AZ routes: False
N reactions shared: 5, N steps in AZ route: 6, in Noctis route: 8
Longest sequence in AZ route: 6, in Noctis route: 6
N branches in AZ route: 0, in Noctis route: 3
2 Exact match Noctis and AZ routes: False
N reactions shared: 6, N steps in AZ route: 6, in Noctis route: 7
Longest sequence in AZ route: 6, in Noctis route: 6
N branches in AZ route: 0, in Noctis route: 2
3 Exact match Noctis and AZ routes: False
N reactions shared: 5, N steps in AZ route: 6, in Noctis route: 7
Longest sequence in AZ route: 6, in Noctis route: 6
N branches in AZ route: 0, in Noctis route: 2
4 Exact match Noctis and AZ routes: False
N reactions shared: 5, N steps in AZ route: 6, in Noctis ro

In [115]:
def create_separator(widths):
    return '+' + '+'.join('-' * (w + 2) for w in widths) + '+'

def create_row(data, widths):
    return '| ' + ' | '.join(f"{d:<{w}}" for d, w in zip(data, widths)) + ' |'

# Define headers
headers = headers = [
    "SMILES",
    "N Routes",
    "Shared Reactions",
    "Steps AZ/Noctis",
    "LongSeq AZ/Noctis",
    "Branches AZ/Noctis",
    "AZ Route recovered",
    "Not AZ Superset"
]

# Prepare the data
table_data = []
for smiles in dict_of_dc.keys():
    if min(shared_reactions[smiles]) == max(shared_reactions[smiles]):
        min_max = str(min(shared_reactions[smiles]))
    else:
        min_max = f'{min(shared_reactions[smiles])}-{max(shared_reactions[smiles])}'
     
    if min(steps_ncts[smiles]) == max(steps_ncts[smiles]):
        stp_az_ncts = f"{str(steps_az[smiles][0])}/{str(min(steps_ncts[smiles]))}"
    else:
        stp_az_ncts = f'{str(steps_az[smiles][0])}/{min(steps_ncts[smiles])}-{max(steps_ncts[smiles])}'
        
    if min(long_seq_ncts[smiles]) == max(long_seq_ncts[smiles]):
        ls_az_ncts = f"{long_seq_az[smiles][0]}/{min(long_seq_ncts[smiles])}"
    else:
        ls_az_ncts = f'{long_seq_az[smiles][0]}/{min(long_seq_ncts[smiles])}-{max(long_seq_ncts[smiles])}'
        
    if min(branch_ncts[smiles]) == max(branch_ncts[smiles]):
        br_az_ncts = f"{branch_az[smiles][0]}/{min(branch_ncts[smiles])}"
    else:
        br_az_ncts = f'{branch_az[smiles][0]}/{min(branch_ncts[smiles])}-{max(branch_ncts[smiles])}'
    
    if len(routes_which_donot_contain_some_reactions[smiles])< max(shared_reactions[smiles]):
        az_recovered = 'Yes'
    else:
        az_recovered = 'No'
    
    row = [
        smiles,
        str(len(dict_of_dc[smiles])),
        min_max,
        stp_az_ncts,
        ls_az_ncts,
        br_az_ncts,
        az_recovered,
        f'{str(len(routes_which_donot_contain_some_reactions[smiles]))}/{str(len(dict_of_dc[smiles]))}'
    ]
    table_data.append(row)

# Calculate column widths
widths = [max(len(str(row[i])) for row in table_data + [headers]) for i in range(len(headers))]

# Print the table
print(create_separator(widths))
print(create_row(headers, widths))
print(create_separator(widths))
for row in table_data:
    print(create_row(row, widths))
print(create_separator(widths))

+--------------------------------------------------------------------------+----------+------------------+-----------------+-------------------+--------------------+--------------------+-----------------+
| SMILES                                                                   | N Routes | Shared Reactions | Steps AZ/Noctis | LongSeq AZ/Noctis | Branches AZ/Noctis | AZ Route recovered | Not AZ Superset |
+--------------------------------------------------------------------------+----------+------------------+-----------------+-------------------+--------------------+--------------------+-----------------+
| Cc1ccc(CC(=O)O)c(O)c1                                                    | 6        | 5-6              | 6/7-8           | 6/6               | 0/3                | True               | 3/6             |
| CCC(O)(/C=C/c1ccc(C(CC)(CC)c2cc(C)c(-c3ccc(CC(=O)O)c(F)c3)c(C)c2)cc1C)CC | 270      | 7                | 7/10-13         | 6/6               | 2/4                | True          